# Data Merging & Integration

## Overview
Combining multiple datasets is a key step in data analysis. This notebook demonstrates how to efficiently merge datasets using different join techniques, ensuring consistency and completeness in the final dataset.

Key steps include:
- Understanding different types of joins (`inner`, `left`, `right`, `outer`)
- Merging datasets based on common keys
- Handling duplicate records after merging
- Checking for inconsistencies in merged datasets

---

## 1 · Imports & Environment Setup
Import standard-library modules, third-party packages, and configure logging and display options.

In [16]:
import os
import sys
import logging
import warnings
from pathlib import Path
from datetime import datetime

import pandas as pd
import chardet
from IPython.display import display

# Suppress non-critical warnings
warnings.filterwarnings('ignore')

# Set up logging with formatting
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('data_merging')

# Force Jupyter Notebook to use all available horizontal space
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set width to a large number
pd.set_option('display.max_colwidth', None) # Show full column content if needed
pd.set_option('display.float_format', '{:,.2f}'.format)  # Format numbers with 2 decimal places

print("✅ Imports complete")

✅ Imports complete


## 2 · Project Folders
Create or validate the input/output directories used throughout the notebook.

In [17]:
def setup_project():
    """Set up project paths and folders"""
    project_root = Path.cwd()  # Current working directory
    input_path = project_root.parent /  '02 Data' / '01_processed_data' / '04_agregated_data'
    output_path = project_root.parent /  '02 Data' / '01_processed_data' / '05_merged_data'
    
    # Check if input directory exists
    if not input_path.exists():
        print(f"Error: Input directory '{input_path}' does not exist.")
        print("Please create this directory or modify the path.")
        sys.exit(1)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    print(f"Project setup complete. \n Input path: {input_path} \n Output path: {output_path}")
    
    summary_report_path = output_path / 'summary_report.txt'
    
    return input_path, output_path, summary_report_path

# Run the setup function
input_path, output_path, summary_report_path = setup_project()

Project setup complete. 
 Input path: C:\Users\User\Dropbox\Personal\CareerFoundry\06 Sourcing data\Notebook folder\02 Data\01_processed_data\04_agregated_data 
 Output path: C:\Users\User\Dropbox\Personal\CareerFoundry\06 Sourcing data\Notebook folder\02 Data\01_processed_data\05_merged_data


## 3 · Helper Functions
Utility functions for encoding detection, safe file loading, and file selection.

In [18]:
def detect_encoding_and_delimiter(file_path):
    """Detect file encoding and best delimiter"""
    # Detect file encoding
    print(f"Detecting encoding for {file_path.name}...")
    with open(file_path, 'rb') as file:
        result = chardet.detect(file.read())
    encoding = result['encoding']
    confidence = result['confidence']
    print(f"Detected encoding: {encoding} (confidence: {confidence:.1%})")
    
    # Analyze delimiter options
    print("\nAnalyzing potential delimiters:\n")
    delimiters = [',', ';', r'\t', '|']  # Raw string for tab to avoid escape issues
    delimiter_options = {}
    for i, delim in enumerate(delimiters, start=1):
        try:
            # Use error_bad_lines=False (or on_bad_lines='skip' in newer pandas) to handle problematic rows
            preview_df = pd.read_csv(file_path, engine='python', encoding=encoding, sep=delim, 
                                    nrows=3, on_bad_lines='skip')
            col_count = len(preview_df.columns)
            delimiter_options[i] = (delim, col_count)
            print(f"{i}: Delimiter '{delim}'\n   Found {col_count} columns")
            print(f"   Preview with option {i}:")
            print(preview_df.head(3))
            print("-" * 80 + "\n")
        except Exception as e:
            print(f"{i}: Error with delimiter '{delim}': {e}")
    
    # Suggest the delimiter with the most columns
    if delimiter_options:
        suggested = max(delimiter_options, key=lambda k: delimiter_options[k][1])
        print(f"Suggested option: {suggested} ('{delimiter_options[suggested][0]}') with {delimiter_options[suggested][1]} columns")
        return encoding, delimiter_options[suggested][0]
    else:
        print("No valid delimiters found. Please check the file format.")
        return encoding, ','  # Default to comma if nothing else works

def safe_load_csv(file_path, encoding=None, delimiter=None):
    """Load CSV file with robust error handling"""
    if encoding is None or delimiter is None:
        encoding, delimiter = detect_encoding_and_delimiter(file_path)
    
    print(f"Loading file with encoding: {encoding}, delimiter: '{delimiter}'")
    
    try:
        # Use default quoting (QUOTE_MINIMAL) to respect quoted fields
        df = pd.read_csv(file_path, encoding=encoding, sep=delimiter, 
                         on_bad_lines='skip')
        print(f"\n✅ Loaded {df.shape[0]} rows × {df.shape[1]} columns from {file_path.name}")
        return df
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        print("Trying alternative approach...")
        try:
            # Alternative approach with Python engine which is more flexible
            df = pd.read_csv(file_path, encoding=encoding, sep=delimiter, 
                            engine='python', on_bad_lines='skip')
            print(f"\n✅ Loaded {df.shape[0]} rows × {df.shape[1]} columns from {file_path.name}")
            return df
        except Exception as e2:
            print(f"❌ Alternative approach also failed: {e2}")
            return None

def list_and_select_files(input_path):
    """List available files and let user select which ones to process"""
    # List available files in the input directory
    available_files = list(input_path.glob("*.*"))
    available_files = [f for f in available_files if f.suffix.lower() in ['.csv', '.xlsx', '.xls', '.pkl']]
    
    if not available_files:
        print(f"No supported files found in {input_path}")
        sys.exit(1)
    
    print("Available files for data merging:")
    for idx, f in enumerate(available_files, start=1):
        print(f"{idx}. {f.name}")
    
    file_numbers_input = input(
        "\nEnter the file numbers to process (comma-separated), or leave blank to process all files: "
    ).strip()
    
    if file_numbers_input:
        try:
            indices = [int(num.strip()) for num in file_numbers_input.split(',') if num.strip()]
            # Validate indices and build the list of selected files.
            selected_files = [available_files[i-1] for i in indices if 1 <= i <= len(available_files)]
            if not selected_files:
                print("No valid file numbers were entered.")
        except ValueError:
            print("Error: Please enter valid numbers separated by commas.")
            selected_files = []
    else:
        selected_files = available_files

    print("\nFiles selected for processing:", [f.name for f in selected_files])
    return selected_files

## 4 · Load Data
Read the selected files into a dictionary of DataFrames and display a brief overview of each.

In [19]:
# List and select files to process
selected_files = list_and_select_files(input_path)

# Dictionary to store loaded DataFrames
dataframes = {}

# Load each selected file with robust error handling
for file_path in selected_files:
    file_name = file_path.name
    
    if file_path.suffix.lower() == '.csv':
        # For CSV files, use our robust loading function
        df = safe_load_csv(file_path)
        
    elif file_path.suffix.lower() in ['.xlsx', '.xls']:
        # For Excel files
        print(f"Loading Excel file: {file_name}")
        try:
            df = pd.read_excel(file_path)
            print(f"✅ Loaded {df.shape[0]} rows × {df.shape[1]} columns from {file_name}")
        except Exception as e:
            print(f"❌ Error loading Excel file: {e}")
            df = None
            
    elif file_path.suffix.lower() == '.pkl':
        # For pickle files
        print(f"Loading pickle file: {file_name}")
        try:
            df = pd.read_pickle(file_path)
            print(f"✅ Loaded {df.shape[0]} rows × {df.shape[1]} columns from {file_name}")
        except Exception as e:
            print(f"❌ Error loading pickle file: {e}")
            df = None
    
    else:
        print(f"Unsupported file format: {file_name}")
        df = None
    
    if df is not None:
        # Store in our dictionary of DataFrames
        dataframes[file_name] = df
        
        # Display basic info about the DataFrame
        print(f"\nDataFrame overview for {file_name}:")
        print(f"Shape: {df.shape}")
        print(f"Columns: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
        print("Sample data:")
        display(df.head(3))
        print("-" * 80)

# Check if we successfully loaded any DataFrames
if not dataframes:
    print("No files were successfully loaded. Cannot proceed with merging.")
    sys.exit(1)
    
print(f"Successfully loaded {len(dataframes)} DataFrames")

Available files for data merging:
1. 03_1_depot_centered_clusters_aggregated.csv
2. 04_optimo_routes.xlsx
3. work_time_and_km_clean_aggregated.csv



Enter the file numbers to process (comma-separated), or leave blank to process all files:  1,3



Files selected for processing: ['03_1_depot_centered_clusters_aggregated.csv', 'work_time_and_km_clean_aggregated.csv']
Detecting encoding for 03_1_depot_centered_clusters_aggregated.csv...
Detected encoding: utf-8 (confidence: 99.0%)

Analyzing potential delimiters:

1: Delimiter ','
   Found 30 columns
   Preview with option 1:
   ABS Custumer no  Route Number                                   Full address Service  DeliveryQty  Net Weight  latitude  longitude                                   formatted_address  geocode_confidence  cluster_id    cluster_name  distance_to_depot_km  depot_latitude  depot_longitude                                     depot_formatted_address  depot_geocode_confidence  main_route_no delivery_weekday  service_time_min  total_net_weight_per_route  total_distance_per_route  total_time_per_route  delivery_time_window  route_id  route_position  route_distance_from_last_stop  travel_time_from_last_stop  new_total_distance_per_route  new_total_time_per_route
0  

,ABS Custumer no,Route Number,Full address,Service,DeliveryQty,Net Weight,latitude,longitude,formatted_address,geocode_confidence,cluster_id,cluster_name,distance_to_depot_km,depot_latitude,depot_longitude,depot_formatted_address,depot_geocode_confidence,main_route_no,delivery_weekday,service_time_min,total_net_weight_per_route,total_distance_per_route,total_time_per_route,delivery_time_window,route_id,route_position,route_distance_from_last_stop,travel_time_from_last_stop,new_total_distance_per_route,new_total_time_per_route
0,"4,498,760.00","5,025.00","Toomkivi tn 2, Kiili, 75401 Harju maakond",MAS,1.00,6.10,59.31,24.83,"Toomkivi tn 2, Kiili, 75401 Harju maakond, Estonia",1.00,1,Production LOO,15.70,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,2,Friday,5.79,568.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"3,760,390.00","2,022.00","Ilunurme tee 6, Liivamäe, 74207 Harju maakond",MAS,4.00,11.20,59.43,24.97,"Ilunurme tee 6, Liivamäe, 74207 Harju maakond",1.00,1,Production LOO,1.84,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,2,Tuesday,10.64,557.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"94,154,649.00","2,432.00",Pärnu mnt T4/9330 TALLINN,MAS,1.00,6.10,59.42,24.74,"Vana-Lõuna tn 30, 10132 Tallinn, Estonia",1.00,1,Production LOO,11.60,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,43,Tuesday,5.79,488.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


--------------------------------------------------------------------------------
Detecting encoding for work_time_and_km_clean_aggregated.csv...
Detected encoding: ascii (confidence: 100.0%)

Analyzing potential delimiters:

1: Delimiter ','
   Found 12 columns
   Preview with option 1:
         Date  Year  Month  Day  Route  Route_id           Start_time             end_time  time  distance  time_sum  distance_sum
0  2025-03-17  2025      3   17    102         2  2025-05-16 09:00:00  2025-05-16 19:25:00 10.42    115.00     47.77        361.73
1  2025-03-18  2025      3   18    202         2  2025-05-16 09:34:00  2025-05-16 19:02:00  9.47     53.19     47.77        361.73
2  2025-03-19  2025      3   19    302         2  2025-05-16 08:22:00  2025-05-16 17:36:00  9.23     49.32     47.77        361.73
--------------------------------------------------------------------------------

2: Delimiter ';'
   Found 1 columns
   Preview with option 2:
    Date,Year,Month,Day,Route,Route_id,Start

,Date,Year,Month,Day,Route,Route_id,Start_time,end_time,time,distance,time_sum,distance_sum
0,2025-03-17,2025,3,17,102,2,2025-05-16 09:00:00,2025-05-16 19:25:00,10.42,115.00,47.77,361.73
1,2025-03-18,2025,3,18,202,2,2025-05-16 09:34:00,2025-05-16 19:02:00,9.47,53.19,47.77,361.73
2,2025-03-19,2025,3,19,302,2,2025-05-16 08:22:00,2025-05-16 17:36:00,9.23,49.32,47.77,361.73


--------------------------------------------------------------------------------
Successfully loaded 2 DataFrames


## Cell 5: Select Operation Type (Merge or Concatenate)
This cell dispatches to either  
1. **merge** – join on selected key columns with a user-specified join type, or  
2. **concatenate** – stack DataFrames vertically (row bind).  

It performs basic validation, optional deduplication, runs the operation,
and assigns the result to `result_df`.

In [20]:
# Prompt user to choose operation type
print("\nAvailable operations:")
print("1. merge - Join DataFrames based on common key columns")
print("2. concatenate - Stack DataFrames on top of each other")

while True:
    operation_type_input = input("\nChoose operation type (1 for merge, 2 for concatenate): ").strip()
    
    if operation_type_input == "1":
        operation_type = "merge"
        break
    elif operation_type_input == "2":
        operation_type = "concatenate"
        break
    else:
        print("Invalid choice. Please enter 1 or 2.")

print(f"Selected operation: {operation_type}")


Available operations:
1. merge - Join DataFrames based on common key columns
2. concatenate - Stack DataFrames on top of each other



Choose operation type (1 for merge, 2 for concatenate):  1


Selected operation: merge


## 6 · Execute Merge / Concatenate
Apply the chosen operation and store the result in `result_df`.

In [21]:
result_df = None

if operation_type == "merge":
    # Need at least 2 DataFrames for merging
    if len(dataframes) < 2:
        print("Need at least 2 DataFrames for a merge operation.")
        sys.exit(1)
    
    # Display the first few rows of each DataFrame to help the user identify key columns
    print("\nAvailable DataFrames for merging:")
    for name, df in dataframes.items():
        # print(f"\n{name} - Columns: {', '.join(df.columns)}") check and then delete
        display(df.head(2))
    
    # Get list of DataFrame names for selection
    df_names = list(dataframes.keys())
    
    # Let the user select the first DataFrame
    print("\nAvailable DataFrames:")
    for i, name in enumerate(df_names, 1):
        print(f"{i}. {name}")
    
    while True:
        try:
            left_idx = int(input("\nSelect the first (left) DataFrame by number: ").strip()) - 1
            if 0 <= left_idx < len(df_names):
                left_name = df_names[left_idx]
                break
            print(f"Please enter a number between 1 and {len(df_names)}")
        except ValueError:
            print("Please enter a valid number")
    
    # Let the user select the second DataFrame
    remaining_df_names = [name for i, name in enumerate(df_names) if i != left_idx]
    print("\nAvailable DataFrames to merge with:")
    for i, name in enumerate(remaining_df_names, 1):
        print(f"{i}. {name}")
    
    while True:
        try:
            right_idx = int(input("\nSelect the second (right) DataFrame by number: ").strip()) - 1
            if 0 <= right_idx < len(remaining_df_names):
                right_name = remaining_df_names[right_idx]
                break
            print(f"Please enter a number between 1 and {len(remaining_df_names)}")
        except ValueError:
            print("Please enter a valid number")
    
    left_df = dataframes[left_name]
    right_df = dataframes[right_name]
    
    # Display columns from both DataFrames with numbers
    print(f"\nColumns in {left_name}:")
    for i, col in enumerate(left_df.columns, 1):
        print(f"{i}. {col}")
    
    print(f"\nColumns in {right_name}:")
    for i, col in enumerate(right_df.columns, 1):
        print(f"{i}. {col}")
    
    # Ask for common key columns in left DataFrame by number (MODIFIED FOR MULTIPLE COLUMNS)
    while True:
        left_key_input = input(f"\nEnter numbers of key columns from {left_name} (comma-separated): ").strip()
        try:
            left_key_indices = [int(idx.strip()) - 1 for idx in left_key_input.split(',')]
            if all(0 <= idx < len(left_df.columns) for idx in left_key_indices) and left_key_indices:
                left_keys = [left_df.columns[i] for i in left_key_indices]
                print(f"Selected: {', '.join(left_keys)}")
                break
            else:
                print(f"Please enter valid numbers between 1 and {len(left_df.columns)}")
        except ValueError:
            print("Please enter valid numbers separated by commas")
    
    # Ask for common key columns in right DataFrame by number (MODIFIED FOR MULTIPLE COLUMNS)
    while True:
        right_key_input = input(f"\nEnter numbers of key columns from {right_name} (comma-separated): ").strip()
        try:
            right_key_indices = [int(idx.strip()) - 1 for idx in right_key_input.split(',')]
            if all(0 <= idx < len(right_df.columns) for idx in right_key_indices) and right_key_indices:
                right_keys = [right_df.columns[i] for i in right_key_indices]
                print(f"Selected: {', '.join(right_keys)}")
                
                # Check if number of keys match
                if len(left_keys) != len(right_keys):
                    print(f"⚠️ Warning: Number of keys doesn't match. {len(left_keys)} columns selected for left DataFrame vs {len(right_keys)} for right DataFrame.")
                    confirm = input("Do you still want to proceed? (y/n): ").strip().lower()
                    if confirm != 'y':
                        continue
                break
            else:
                print(f"Please enter valid numbers between 1 and {len(right_df.columns)}")
        except ValueError:
            print("Please enter valid numbers separated by commas")
    
    # Ask for merge type
    print("\nAvailable merge types:")
    print("1. inner - Keep only rows with matching keys in both DataFrames")
    print("2. outer - Keep all rows from both DataFrames")
    print("3. left - Keep all rows from left DataFrame, add matching rows from right DataFrame")
    print("4. right - Keep all rows from right DataFrame, add matching rows from left DataFrame")
    
    merge_types = {1: "inner", 2: "outer", 3: "left", 4: "right"}
    
    while True:
        try:
            merge_type_idx = int(input("\nSelect merge type (1-4): ").strip())
            if 1 <= merge_type_idx <= 4:
                merge_type = merge_types[merge_type_idx]
                break
            print("Please enter a number between 1 and 4")
        except ValueError:
            print("Please enter a valid number")
    
    # Data type check and conversion for multiple key columns
    need_conversion = False
    conversion_pairs = []
    
    for i, (left_key, right_key) in enumerate(zip(left_keys, right_keys)):
        left_dtype = left_df[left_key].dtype
        right_dtype = right_df[right_key].dtype
        
        if left_dtype != right_dtype:
            need_conversion = True
            conversion_pairs.append((i, left_key, right_key, left_dtype, right_dtype))
    
    if need_conversion:
        # Data type conversion code...
        print(f"✅ Conversion complete")
    
    # ===== INSERT OPTION 1 CODE RIGHT HERE =====
    print("\nChecking for duplicate keys in right dataframe...")
    right_key_counts = right_df.groupby(right_keys).size().reset_index(name='count')
    duplicate_keys = right_key_counts[right_key_counts['count'] > 1]

    if len(duplicate_keys) > 0:
        print(f"⚠️ Found {len(duplicate_keys)} keys in the right dataframe that appear multiple times.")
        print(f"This could result in {len(left_df)} rows from left dataframe expanding to potentially more rows in the result.")
        
        dedup_choice = input("\nHow would you like to handle duplicate keys in the right dataframe?\n"
                            "1. Keep first occurrence only\n"
                            "2. Keep last occurrence only\n"
                            "3. Aggregate values (combine duplicate rows)\n"
                            "4. Keep all duplicates (may cause row multiplication)\n"
                            "Enter choice (1-4): ").strip()
        
        if dedup_choice == '1':
            # Keep only the first occurrence of each key
            right_df = right_df.drop_duplicates(subset=right_keys, keep='first')
            print(f"✅ Right dataframe deduplicated to {len(right_df)} rows (kept first occurrence)")
            
        elif dedup_choice == '2':
            # Keep only the last occurrence of each key
            right_df = right_df.drop_duplicates(subset=right_keys, keep='last')
            print(f"✅ Right dataframe deduplicated to {len(right_df)} rows (kept last occurrence)")
            
        elif dedup_choice == '3':
            # Aggregate values for each key
            print("\nFor duplicate keys, how should non-key columns be aggregated?")
            print("1. First value")
            print("2. Last value")
            print("3. Mean (numeric columns only)")
            print("4. Sum (numeric columns only)")
            print("5. Custom (specify for each column)")
            
            agg_method = input("Enter choice (1-5): ").strip()
            
            if agg_method == '5':
                # Custom aggregation for each column
                agg_dict = {}
                non_key_cols = [col for col in right_df.columns if col not in right_keys]
                
                print("\nFor each column, specify aggregation method:")
                print("f = first, l = last, m = mean, s = sum, x = max, n = min")
                
                for col in non_key_cols:
                    col_method = input(f"{col} (f/l/m/s/x/n): ").strip().lower()
                    
                    if col_method == 'f':
                        agg_dict[col] = 'first'
                    elif col_method == 'l':
                        agg_dict[col] = 'last'
                    elif col_method == 'm':
                        agg_dict[col] = 'mean'
                    elif col_method == 's':
                        agg_dict[col] = 'sum'
                    elif col_method == 'x':
                        agg_dict[col] = 'max'
                    elif col_method == 'n':
                        agg_dict[col] = 'min'
                    else:
                        agg_dict[col] = 'first'  # Default
                        
                right_df = right_df.groupby(right_keys).agg(agg_dict).reset_index()
                
            else:
                # Apply the same aggregation method to all non-key columns
                agg_method_map = {'1': 'first', '2': 'last', '3': 'mean', '4': 'sum'}
                agg_func = agg_method_map.get(agg_method, 'first')
                
                right_df = right_df.groupby(right_keys).agg(agg_func).reset_index()
                
            print(f"✅ Right dataframe aggregated to {len(right_df)} rows")
        
        elif dedup_choice == '4':
            print("Proceeding with merge keeping all duplicates.")
            print("Note: This may cause rows in left dataframe to appear multiple times in result.")
        
        else:
            print("Invalid choice. Proceeding with all duplicates.")
    
    # After the conversion, proceed with the merge operation
    print(f"\nPerforming {merge_type} merge of {left_name} with {right_name}")
    print(f"Using keys: {', '.join([f'{lk}={rk}' for lk, rk in zip(left_keys, right_keys)])}")
    
    # Perform the merge with indicator to track which rows came from where
    try:
        result_df = left_df.merge(
            right_df, 
            left_on=left_keys,  # Now using lists of keys 
            right_on=right_keys, # Now using lists of keys
            how=merge_type,
            indicator=True
        )
        print(f"✅ Merge successful! Result has {result_df.shape[0]} rows and {result_df.shape[1]} columns")
    except Exception as e:
        print(f"❌ Merge operation failed: {e}")
        print("\nAttempting alternative approach with reset index...")
        try:
            # Try with reset index - sometimes helps with complex index structures
            left_df_reset = left_df.reset_index(drop=True)
            right_df_reset = right_df.reset_index(drop=True)
            
            result_df = left_df_reset.merge(
                right_df_reset, 
                left_on=left_keys, 
                right_on=right_keys,
                how=merge_type,
                indicator=True
            )
            print(f"✅ Alternative merge successful! Result has {result_df.shape[0]} rows and {result_df.shape[1]} columns")
        except Exception as e2:
            print(f"❌ Alternative approach also failed: {e2}")
            print("Please check your data or try different key columns.")
            result_df = None
    
    # Calculate and display merge statistics
    if result_df is not None and '_merge' in result_df.columns:
        merge_counts = result_df['_merge'].value_counts()
        merge_percentages = result_df['_merge'].value_counts(normalize=True) * 100
        
        print("\nMerge Results:")
        print("-" * 40)
        print("\nMerge Statistics:")
        merge_stats = pd.DataFrame({
            'Count': merge_counts, 
            'Percentage (%)': merge_percentages
        })
        print(merge_stats)
        
        print("\nSample of merged data:")
        display(result_df.head())
        
        # Ask if user wants to keep the _merge column
        keep_merge = input("\nKeep the '_merge' column for analysis? (y/n): ").strip().lower()
        if keep_merge != 'y':
            result_df = result_df.drop(columns=['_merge'])

elif operation_type == "concatenate":
    # Use all the DataFrames that were already loaded
    print(f"\nConcatenating all {len(dataframes)} loaded DataFrames")
    
    # Ask about axis and index reset
    axis = 0  # Default to row-wise concatenation
    axis_choice = input("\nConcatenate row-wise (0) or column-wise (1)? [0/1]: ").strip()
    if axis_choice == '1':
        axis = 1
        
    reset_index = False
    reset_choice = input("\nReset indices? [y/n]: ").strip().lower()
    if reset_choice == 'y':
        reset_index = True
    
    # Perform the concatenation
    try:
        selected_dfs = list(dataframes.values())
        selected_names = list(dataframes.keys())
        result_df = pd.concat(selected_dfs, axis=axis, ignore_index=reset_index)
        print(f"\n✅ Concatenation successful! Result has {result_df.shape[0]} rows and {result_df.shape[1]} columns")
    except Exception as e:
        print(f"❌ Concatenation operation failed: {e}")
        result_df = None

# Check result and display summary information
if result_df is not None:
    print("\nOperation completed successfully")
    print(f"Result shape: {result_df.shape[0]} rows × {result_df.shape[1]} columns")
    # ... (rest of your original code)
else:
    print("Operation failed or resulted in an empty DataFrame")


Available DataFrames for merging:


,ABS Custumer no,Route Number,Full address,Service,DeliveryQty,Net Weight,latitude,longitude,formatted_address,geocode_confidence,cluster_id,cluster_name,distance_to_depot_km,depot_latitude,depot_longitude,depot_formatted_address,depot_geocode_confidence,main_route_no,delivery_weekday,service_time_min,total_net_weight_per_route,total_distance_per_route,total_time_per_route,delivery_time_window,route_id,route_position,route_distance_from_last_stop,travel_time_from_last_stop,new_total_distance_per_route,new_total_time_per_route
0,"4,498,760.00","5,025.00","Toomkivi tn 2, Kiili, 75401 Harju maakond",MAS,1.00,6.10,59.31,24.83,"Toomkivi tn 2, Kiili, 75401 Harju maakond, Estonia",1.00,1,Production LOO,15.70,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,2,Friday,5.79,568.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"3,760,390.00","2,022.00","Ilunurme tee 6, Liivamäe, 74207 Harju maakond",MAS,4.00,11.20,59.43,24.97,"Ilunurme tee 6, Liivamäe, 74207 Harju maakond",1.00,1,Production LOO,1.84,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,2,Tuesday,10.64,557.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Date,Year,Month,Day,Route,Route_id,Start_time,end_time,time,distance,time_sum,distance_sum
0,2025-03-17,2025,3,17,102,2,2025-05-16 09:00:00,2025-05-16 19:25:00,10.42,115.00,47.77,361.73
1,2025-03-18,2025,3,18,202,2,2025-05-16 09:34:00,2025-05-16 19:02:00,9.47,53.19,47.77,361.73



Available DataFrames:
1. 03_1_depot_centered_clusters_aggregated.csv
2. work_time_and_km_clean_aggregated.csv



Select the first (left) DataFrame by number:  1


...


How would you like to handle duplicate keys in the right dataframe?
1. Keep first occurrence only
2. Keep last occurrence only
3. Aggregate values (combine duplicate rows)
4. Keep all duplicates (may cause row multiplication)
Enter choice (1-4):  1


✅ Right dataframe deduplicated to 4 rows (kept first occurrence)

Performing left merge of 03_1_depot_centered_clusters_aggregated.csv with work_time_and_km_clean_aggregated.csv
Using keys: main_route_no=Route_id
✅ Merge successful! Result has 885 rows and 43 columns

Merge Results:
----------------------------------------

Merge Statistics:
            Count  Percentage (%)
_merge                           
both          885          100.00
left_only       0            0.00
right_only      0            0.00

Sample of merged data:


,ABS Custumer no,Route Number,Full address,Service,DeliveryQty,Net Weight,latitude,longitude,formatted_address,geocode_confidence,cluster_id,cluster_name,distance_to_depot_km,depot_latitude,depot_longitude,depot_formatted_address,depot_geocode_confidence,main_route_no,delivery_weekday,service_time_min,total_net_weight_per_route,total_distance_per_route,total_time_per_route,delivery_time_window,route_id,route_position,route_distance_from_last_stop,travel_time_from_last_stop,new_total_distance_per_route,new_total_time_per_route,Date,Year,Month,Day,Route,Route_id,Start_time,end_time,time,distance,time_sum,distance_sum,_merge
0,"4,498,760.00","5,025.00","Toomkivi tn 2, Kiili, 75401 Harju maakond",MAS,1.00,6.10,59.31,24.83,"Toomkivi tn 2, Kiili, 75401 Harju maakond, Estonia",1.00,1,Production LOO,15.70,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,2,Friday,5.79,568.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-17,2025,3,17,102,2,2025-05-16 09:00:00,2025-05-16 19:25:00,10.42,115.00,47.77,361.73,both
1,"3,760,390.00","2,022.00","Ilunurme tee 6, Liivamäe, 74207 Harju maakond",MAS,4.00,11.20,59.43,24.97,"Ilunurme tee 6, Liivamäe, 74207 Harju maakond",1.00,1,Production LOO,1.84,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,2,Tuesday,10.64,557.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-17,2025,3,17,102,2,2025-05-16 09:00:00,2025-05-16 19:25:00,10.42,115.00,47.77,361.73,both
2,"94,154,649.00","2,432.00",Pärnu mnt T4/9330 TALLINN,MAS,1.00,6.10,59.42,24.74,"Vana-Lõuna tn 30, 10132 Tallinn, Estonia",1.00,1,Production LOO,11.60,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,43,Tuesday,5.79,488.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-17,2025,3,17,143,43,2025-05-16 13:30:00,2025-05-16 17:15:00,3.75,31.00,31.11,274.00,both
3,"3,940,965.00","5,025.00","Valli tee 17, Vaela, 75413 Harju maakond",MAS,2.00,10.10,59.34,24.80,"Valli tee 17, Vaela, 75413 Harju maakond, Estonia",1.00,1,Production LOO,14.11,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,2,Friday,9.59,568.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-17,2025,3,17,102,2,2025-05-16 09:00:00,2025-05-16 19:25:00,10.42,115.00,47.77,361.73,both
4,"94,151,273.00","5,025.00","Suurvälja tee 5, Vaela, 75413 Harju maakond",MAS,1.00,0.00,59.33,24.80,"Suurvälja tee 5, Vaela, 75413 Harju maakond, Estonia",1.00,1,Production LOO,14.11,59.44,24.94,"Kuusiku tee 26, Loo, Jõelähtme, 74201 Harju Maakond, Eesti",0.99,2,Friday,1.00,568.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-17,2025,3,17,102,2,2025-05-16 09:00:00,2025-05-16 19:25:00,10.42,115.00,47.77,361.73,both



Keep the '_merge' column for analysis? (y/n):  n



Operation completed successfully
Result shape: 885 rows × 42 columns


## 7 · Save Results & Summary
Persist the merged DataFrame to disk and write a summary report outlining key statistics.

In [22]:
if result_df is not None:
    # Ask user if they want to drop any columns
    print("\nThe merged dataset contains the following columns:")
    for i, col in enumerate(result_df.columns, 1):
        print(f"{i}. {col}")
    
    drop_columns = input("\nWould you like to drop any columns? (y/n): ").strip().lower()
    
    if drop_columns == 'y':
        while True:
            columns_to_drop_input = input("\nEnter column numbers to drop (comma-separated), or 'done' to finish: ").strip()
            
            if columns_to_drop_input.lower() == 'done':
                break
                
            try:
                # Parse as column numbers
                if all(item.strip().isdigit() for item in columns_to_drop_input.split(',')):
                    indices = [int(idx.strip()) - 1 for idx in columns_to_drop_input.split(',')]
                    selected_columns = [result_df.columns[i] for i in indices if 0 <= i < len(result_df.columns)]
                    
                    if selected_columns:
                        # Confirm with user
                        print(f"\nYou selected to drop these columns: {', '.join(selected_columns)}")
                        confirm = input("Confirm? (y/n): ").strip().lower()
                        
                        if confirm == 'y':
                            result_df = result_df.drop(columns=selected_columns)
                            print(f"✅ Dropped {len(selected_columns)} columns. {len(result_df.columns)} columns remaining.")
                            
                            # Display remaining columns
                            print("\nRemaining columns:")
                            for i, col in enumerate(result_df.columns, 1):
                                print(f"{i}. {col}")
                        else:
                            print("No columns were dropped.")
                    else:
                        print("No valid columns were selected.")
                else:
                    print("Please enter valid column numbers separated by commas.")
            except Exception as e:
                print(f"Error: {e}. Please try again.")
    
    # Ask for file format preference
    print("\nAvailable file formats for saving:")
    print("1. CSV (.csv)")
    print("2. Excel (.xlsx)")
    print("3. Pickle (.pkl)")
    
    while True:
        file_format_choice = input("\nChoose file format (1-3): ").strip()
        if file_format_choice in ['1', '2', '3']:
            break
        print("Please enter a number between 1 and 3")
    
    format_map = {'1': 'csv', '2': 'xlsx', '3': 'pkl'}
    file_format = format_map[file_format_choice]
    
    # Ask for output file name
    default_name = f"merged_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    output_filename = input(f"\nEnter the desired file name without extension [default: {default_name}]: ").strip()
    if not output_filename:
        output_filename = default_name
    
    # Create full file path
    output_file = output_path / f"{output_filename}.{file_format}"
    
    # Save the result
    try:
        if file_format == 'csv':
            result_df.to_csv(output_file, index=False)
        elif file_format == 'xlsx':
            result_df.to_excel(output_file, index=False)
        elif file_format == 'pkl':
            result_df.to_pickle(output_file)
        
        print(f"\n✅ Result saved to: {output_file}")
        
        # Generate summary report
        with open(summary_report_path, 'w') as report:
            report.write("Data Processing Report\n")
            report.write("============================\n")
            report.write(f"Date and time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            if operation_type == 'merge':
                report.write(f"Operation: Merge ({merge_type})\n")
                report.write(f"Left DataFrame: {left_name}\n")
                report.write(f"Right DataFrame: {right_name}\n")
                
                # Updated for multiple keys
                key_pairs = [f"{lk}={rk}" for lk, rk in zip(left_keys, right_keys)]
                report.write(f"Merge keys: {', '.join(key_pairs)}\n\n")
                
                # Write merge statistics if available
                if '_merge' in result_df.columns:
                    report.write("Merge Statistics:\n")
                    report.write(str(merge_stats) + "\n\n")
            else:
                report.write(f"Operation: Concatenation\n")
                report.write(f"DataFrames used: {', '.join(selected_names)}\n")
                report.write(f"Reset index: {reset_index}\n\n")
            
            report.write(f"Result shape: {result_df.shape[0]} rows × {result_df.shape[1]} columns\n")
            report.write(f"Result columns: {', '.join(result_df.columns)}\n\n")
            
            # Write missing value information
            na_count = result_df.isna().sum().sum()
            report.write(f"Missing values: {na_count}\n")
            if na_count > 0:
                na_cols = result_df.columns[result_df.isna().any()].tolist()
                report.write(f"Columns with missing values: {', '.join(na_cols)}\n")
            
            report.write(f"\nOutput file: {output_file}\n")
        
        print(f"Summary report saved to: {summary_report_path}")
        
    except Exception as e:
        print(f"❌ Error saving file: {e}")
else:
    print("No data to save")


The merged dataset contains the following columns:
1. ABS Custumer no
2. Route Number
3. Full address
4. Service
5. DeliveryQty
6. Net Weight
7. latitude
8. longitude
9. formatted_address
10. geocode_confidence
11. cluster_id
12. cluster_name
13. distance_to_depot_km
14. depot_latitude
15. depot_longitude
16. depot_formatted_address
17. depot_geocode_confidence
18. main_route_no
19. delivery_weekday
20. service_time_min
21. total_net_weight_per_route
22. total_distance_per_route
23. total_time_per_route
24. delivery_time_window
25. route_id
26. route_position
27. route_distance_from_last_stop
28. travel_time_from_last_stop
29. new_total_distance_per_route
30. new_total_time_per_route
31. Date
32. Year
33. Month
34. Day
35. Route
36. Route_id
37. Start_time
38. end_time
39. time
40. distance
41. time_sum
42. distance_sum



Would you like to drop any columns? (y/n):  y

Enter column numbers to drop (comma-separated), or 'done' to finish:  31,32,33,34,35,36,37,38,39,40



You selected to drop these columns: Date, Year, Month, Day, Route, Route_id, Start_time, end_time, time, distance


Confirm? (y/n):  y


✅ Dropped 10 columns. 32 columns remaining.

Remaining columns:
1. ABS Custumer no
2. Route Number
3. Full address
4. Service
5. DeliveryQty
6. Net Weight
7. latitude
8. longitude
9. formatted_address
10. geocode_confidence
11. cluster_id
12. cluster_name
13. distance_to_depot_km
14. depot_latitude
15. depot_longitude
16. depot_formatted_address
17. depot_geocode_confidence
18. main_route_no
19. delivery_weekday
20. service_time_min
21. total_net_weight_per_route
22. total_distance_per_route
23. total_time_per_route
24. delivery_time_window
25. route_id
26. route_position
27. route_distance_from_last_stop
28. travel_time_from_last_stop
29. new_total_distance_per_route
30. new_total_time_per_route
31. time_sum
32. distance_sum



Enter column numbers to drop (comma-separated), or 'done' to finish:  done



Available file formats for saving:
1. CSV (.csv)
2. Excel (.xlsx)
3. Pickle (.pkl)



Choose file format (1-3):  1

Enter the desired file name without extension [default: merged_data_20250516_153712]:  



✅ Result saved to: C:\Users\User\Dropbox\Personal\CareerFoundry\06 Sourcing data\Notebook folder\02 Data\01_processed_data\05_merged_data\merged_data_20250516_153712.csv
Summary report saved to: C:\Users\User\Dropbox\Personal\CareerFoundry\06 Sourcing data\Notebook folder\02 Data\01_processed_data\05_merged_data\summary_report.txt
